In [27]:
import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint
import torch.nn as nn
import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms 
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [54]:
Re = 400
bs = 16
epochs = 10

In [55]:
class CustomDatasetUV(Dataset):
    def __init__(self, root_dir, data_transform=None, label_transform=None):
        self.root_dir = root_dir
        self.data_transform = data_transform
        self.label_transform = label_transform
        self.image_paths = []
        self.labels_path = []
        image_dir = os.path.join(root_dir, 'Mesh')
        u_labels_dir = os.path.join(root_dir, 'W_Data')
        v_labels_dir = os.path.join(root_dir, 'W_Data')
        for file in os.listdir(image_dir):
            Ulabel_file = 'V' + file[4:]
            Vlabel_file = 'U' + file[4:]
            if file.endswith('.png'):
                self.image_paths.append(os.path.join(image_dir, file))
                self.labels_path.append((os.path.join(u_labels_dir, Ulabel_file), os.path.join(v_labels_dir, Vlabel_file)))


    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label_path = self.labels_path[idx]
        image = Image.open(img_path).convert('L')
        Ulabel = Image.open(label_path[0])
        Ulabel = Ulabel.transpose(Image.FLIP_TOP_BOTTOM)
        Ulabel = Ulabel.transpose(Image.FLIP_LEFT_RIGHT)
        Vlabel = Image.open(label_path[1])
        Vlabel = Vlabel.transpose(Image.FLIP_TOP_BOTTOM)
        Vlabel = Vlabel.transpose(Image.FLIP_LEFT_RIGHT)

        image = self.data_transform(image)
        Ulabel = self.label_transform(Ulabel)
        Vlabel = self.label_transform(Vlabel)
        label = torch.concat((Ulabel, Vlabel), dim=0)
        return image, label
    
class CustomDatasetMag(Dataset):
    def __init__(self, root_dir, data_transform=None, label_transform=None):
        self.root_dir = root_dir
        self.data_transform = data_transform
        self.label_transform = label_transform
        self.image_paths = []
        self.labels_path = []
        image_dir = os.path.join(root_dir, 'Mesh')
        labels_dir = os.path.join(root_dir, 'W_Data')
        for file in os.listdir(image_dir):
            label_file = 'U' + file[4:]
            if file.endswith('.png'):
                self.image_paths.append(os.path.join(image_dir, file))
                self.labels_path.append(os.path.join(labels_dir, label_file))


    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label_path = self.labels_path[idx]
        image = Image.open(img_path).convert('L')
        label = Image.open(label_path)

        image = self.data_transform(image)
        label = self.label_transform(label)
        return image, label
    
    # Define transformations
img_transform = transforms.Compose([
    transforms.Resize((128, 256)),
    transforms.ToTensor()
])

label_transform = transforms.Compose([
    transforms.Resize((128, 256)),
    transforms.ToTensor()
])
# Path to your dataset folder
dataset_root = ".\\"

# Create dataset
dataset = CustomDatasetMag(dataset_root, data_transform=img_transform, label_transform = label_transform)

# Split dataset into train and validation
dataset_size = len(dataset)
val_split = 0.2
val_size = int(val_split * dataset_size)
train_size = dataset_size - val_size

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [56]:
class ChannelAttention(nn.Module):
    def __init__(self, gate_channels, reduction_ratio=16, pool_types=['avg', 'max']):
        super(ChannelAttention, self).__init__()
        self.gate_channels = gate_channels
        self.pool_types = pool_types
        self.mlp = nn.Sequential(
            nn.Flatten(),
            nn.Linear(gate_channels, gate_channels // reduction_ratio),
            nn.ReLU(),
            nn.Linear(gate_channels // reduction_ratio, gate_channels)
            )
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        channel_att_sum = 0
        for pool_type in self.pool_types:
            if pool_type=='avg':
                avg_pool = self.avg_pool(x)
                channel_att_raw = self.mlp(avg_pool)
            elif pool_type=='max':
                max_pool = self.max_pool(x)
                channel_att_raw = self.mlp(max_pool)

        channel_att_sum = channel_att_sum + channel_att_raw

        scale = self.sigmoid(x).expand_as(x)
        return x * scale

In [57]:
class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()

        assert kernel_size in (3, 7), 'kernel size must be 3 or 7'
        padding = 3 if kernel_size == 7 else 1

        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input):
        avg_out = torch.mean(input, dim=1, keepdim=True)
        max_out, _ = torch.max(input, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return input * self.sigmoid(x)

In [58]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [59]:
def blockUNet(in_c, out_c, name, transposed=False, bn=True, relu=True, size=4, pad=1, dropout=0.):
    block = nn.Sequential()
    if relu:
        block.add_module('%s_relu' % name, nn.ReLU(inplace=True))
    else:
        block.add_module('%s_leakyrelu' % name, nn.LeakyReLU(0.2, inplace=True))
    if not transposed:
        block.add_module('%s_conv' % name, nn.Conv2d(in_c, out_c, kernel_size=size, stride=2, padding=pad, bias=True))
    else:
        block.add_module('%s_upsam' % name, nn.Upsample(scale_factor=2, mode='bilinear')) # Note: old default was nearest neighbor
        # reduce kernel size by one for the upsampling (ie decoder part)
        block.add_module('%s_tconv' % name, nn.Conv2d(in_c, out_c, kernel_size=(size-1), stride=1, padding=pad, bias=True))
    if bn:
        block.add_module('%s_bn' % name, nn.BatchNorm2d(out_c))
    if dropout>0.:
        block.add_module('%s_dropout' % name, nn.Dropout2d( dropout, inplace=True))
    return block

In [60]:
# generator model
class TNet(nn.Module):
    def __init__(self, channelExponent=6, dropout=0.):
        super(TNet, self).__init__()
        channels = int(2 ** channelExponent + 0.5)

        self.layer1 = nn.Sequential()
        self.layer1.add_module('layer1_conv', nn.Conv2d(1, channels, 4, 2, 1, bias=True))

        self.layer2 = blockUNet(channels  , channels*2, 'layer2', transposed=False, bn=True,  relu=False, dropout=dropout )
        self.layer2b= blockUNet(channels*2, channels*2, 'layer2b',transposed=False, bn=True,  relu=False, dropout=dropout )
        self.layer3 = blockUNet(channels*2, channels*4, 'layer3', transposed=False, bn=True,  relu=False, dropout=dropout )
        # note the following layer also had a kernel size of 2 in the original version (cf https://arxiv.org/abs/1810.08217)
        # it is now changed to size 4 for encoder/decoder symmetry; to reproduce the old/original results, please change it to 2
        self.layer4 = blockUNet(channels*4, channels*8, 'layer4', transposed=False, bn=True,  relu=False, dropout=dropout ,  size=4 ) # note, size 4!
        self.layer5 = blockUNet(channels*8, channels*8, 'layer5', transposed=False, bn=True,  relu=False, dropout=dropout , size=2,pad=0)
        self.layer6 = blockUNet(channels*8, channels*8, 'layer6', transposed=False, bn=False, relu=False, dropout=dropout , size=2,pad=0)
     
        # note, kernel size is internally reduced by one now
        self.dlayer6 = blockUNet(channels*8, channels*8, 'dlayer6', transposed=True, bn=True, relu=True, dropout=dropout , size=2,pad=0)
        self.dlayer5 = blockUNet(channels*16,channels*8, 'dlayer5', transposed=True, bn=True, relu=True, dropout=dropout , size=2,pad=0)
        self.dlayer4 = blockUNet(channels*16,channels*4, 'dlayer4', transposed=True, bn=True, relu=True, dropout=dropout ) 
        self.dlayer3 = blockUNet(channels*8, channels*2, 'dlayer3', transposed=True, bn=True, relu=True, dropout=dropout )
        self.dlayer2b= blockUNet(channels*4, channels*2, 'dlayer2b',transposed=True, bn=True, relu=True, dropout=dropout )
        self.dlayer2 = blockUNet(channels*4, channels  , 'dlayer2', transposed=True, bn=True, relu=True, dropout=dropout )

        self.dlayer1 = nn.Sequential()
        self.dlayer1.add_module('dlayer1_relu', nn.ReLU(inplace=True))
        self.dlayer1.add_module('dlayer1_tconv', nn.ConvTranspose2d(channels*2, 3, 4, 2, 1, bias=True))

    def forward(self, x):
        out1 = self.layer1(x)
        out2 = self.layer2(out1)
        out2b= self.layer2b(out2)
        out3 = self.layer3(out2b)
        out4 = self.layer4(out3)
        out5 = self.layer5(out4)
        out6 = self.layer6(out5)
        dout6 = self.dlayer6(out6)
        dout6_out5 = torch.cat([dout6, out5], 1)
        dout5 = self.dlayer5(dout6_out5)
        dout5_out4 = torch.cat([dout5, out4], 1)
        dout4 = self.dlayer4(dout5_out4)
        dout4_out3 = torch.cat([dout4, out3], 1)
        dout3 = self.dlayer3(dout4_out3)
        dout3_out2b = torch.cat([dout3, out2b], 1)
        dout2b = self.dlayer2b(dout3_out2b)
        dout2b_out2 = torch.cat([dout2b, out2], 1)
        dout2 = self.dlayer2(dout2b_out2)
        dout2_out1 = torch.cat([dout2, out1], 1)
        dout1 = self.dlayer1(dout2_out1)
        return dout1

In [61]:
class DataLoss(nn.Module):
    def __init__(self):
        super(DataLoss, self).__init__()
        
    def forward(self, true, pred):
        data_loss = (torch.abs(true - pred).sum()) / (6 * true.shape[0] * true.shape[2] * true.shape[3])
        return data_loss
    
class PhysicsLoss(nn.Module):
    def __init__(self):
        super(PhysicsLoss, self).__init__()
        
    def forward(self, true, pred):
        ui = true[:, 0, 1:-1, 1:-1]
        vi = true[:, 1, 1:-1, 1:-1]
        ut = pred[:, 0, 1:-1, 1:-1]
        vt = pred[:, 1, 1:-1, 1:-1]

        dui_dx = (true[:, 0, 2:, 1:-1] - true[:, 0, :-2, 1:-1] ) / 2
        dui_dy = (true[:, 0, 1:-1:, 2:] - true[:, 0, 1:-1, :-2] ) / 2
        dvi_dy = (true[:, 1, 1:-1, 2:] - true[:, 1, 1:-1, :-2] ) / 2
        dvi_dx = (true[:, 1, 2:, 1:-1] - true[:, 1, :-2, 1:-1] ) / 2
        dut_dx = (pred[:, 0, 2:, 1:-1] - pred[:, 0, :-2, 1:-1] ) / 2
        dut_dy = (pred[:, 0, 1:-1:, 2:] - pred[:, 0, 1:-1, :-2] ) / 2
        dvt_dy = (pred[:, 1, 1:-1, 2:] - pred[:, 1, 1:-1, :-2] ) / 2
        dvt_dx = (pred[:, 1, 2:, 1:-1] - pred[:, 1, :-2, 1:-1] ) / 2

        d2ui_dx2 = (true[:, 0, 2:, 1:-1] - 2 * true[:, 0, 1:-1, 1:-1]  + true[:, 0, :-2, 1:-1]) 
        d2ui_dy2 = (true[:, 0, 1:-1, 2:] - 2 * true[:, 0, 1:-1, 1:-1]  + true[:, 0, 1:-1, :-2]) 
        d2vi_dy2 = (true[:, 1, 1:-1, 2:] - 2 * true[:, 1, 1:-1, 1:-1] + true[:, 1, 1:-1, :-2])
        d2vi_dx2 = (true[:, 1, 2:, 1:-1] - 2 * true[:, 1, 1:-1, 1:-1]  + true[:, 1, :-2, 1:-1])
        d2ut_dx2 = (pred[:, 0, 2:, 1:-1] - 2 * pred[:, 0, 1:-1, 1:-1]  + pred[:, 0, :-2, 1:-1]) 
        d2ut_dy2 = (pred[:, 0, 1:-1, 2:] - 2 * pred[:, 0, 1:-1, 1:-1]  + pred[:, 0, 1:-1, :-2]) 
        d2vt_dy2 = (pred[:, 1, 1:-1, 2:] - 2 * pred[:, 1, 1:-1, 1:-1] + pred[:, 1, 1:-1, :-2])
        d2vt_dx2 = (pred[:, 1, 2:, 1:-1] - 2 * pred[:, 1, 1:-1, 1:-1]  + pred[:, 1, :-2, 1:-1])

        mass_loss = torch.abs((dui_dx + dvi_dy) - (dut_dx + dvt_dy)).sum() / (true.shape[0] * (true.shape[2] - 2) * (true.shape[3] - 2))

        momentum_loss_x = torch.abs(((2 * dui_dx + ui * dvi_dy + vi * dui_dy) - (d2ui_dx2 + d2ui_dy2) / Re) - ((2 * dut_dx + ut * dvt_dy + vt * dut_dy) - (d2ut_dx2 + d2ut_dy2) / Re))
        momentum_loss_y = torch.abs(((2 * dvi_dy + ui * dvi_dx + vi * dui_dx) - (d2vi_dx2 + d2vi_dy2) / Re) - ((2 * dvt_dy + ut * dvt_dx + vt * dut_dx) - (d2vt_dx2 + d2vt_dy2) / Re))

        total_momentum_loss = (momentum_loss_x + momentum_loss_y).sum() / (true.shape[0] * (true.shape[2] - 2) * (true.shape[3] - 2))
        return mass_loss, total_momentum_loss
    
class Loss(nn.Module):
    def __init__(self, alpha_1 = 0.33, alpha_2 = 1.67, alpha_3 = 8.33) -> None:
        super(Loss, self).__init__()
        self.a1 = alpha_1
        self.a2 = alpha_2
        self.a3 = alpha_3
        self.data_loss = DataLoss()
        self.physics_loss = PhysicsLoss()

    def forward(self, true, pred):
        loss_data = self.data_loss(true, pred)
        loss_mass, loss_momentum = self.physics_loss(true, pred)

        total_loss = self.a1 * loss_data + self.a2 * loss_mass + self.a3 * loss_momentum
        return total_loss

In [62]:
def MRE(true, pred):
    return ((1 / true.shape[0]) * ((torch.abs(true - pred)).sum()) / true.sum())

In [63]:
class Tnet_Module(L.LightningModule):
    def __init__(self) -> None:
        super().__init__()
        self.model = TNet()
        self.loss = DataLoss()

    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(), lr=1e-4)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=25, gamma=0.1)
        return [optimizer], [scheduler]
    
    def training_step(self, batch, batch_idx):
        imgs, labels = batch
        preds = self.model(imgs)
        loss = self.loss(labels, preds)
        mre = MRE(labels, preds)

        self.log("train/loss", loss)
        self.log("train/mre", mre)
        return loss
    
    def validation_step(self, batch, batch_idx):
        imgs, labels = batch
        preds = self.model(imgs)
        loss = self.loss(labels, preds)
        mre = MRE(labels, preds)

        self.log("val/loss", loss)
        self.log("val/mre", mre)  

In [65]:
model = Tnet_Module()
checkpoint_callback = ModelCheckpoint(monitor="val/mre", mode="min")
trainer = L.Trainer(max_epochs=10, callbacks=[checkpoint_callback])
trainer.fit(model, train_loader, val_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type     | Params
-----------------------------------
0 | model | TNet     | 9.3 M 
1 | loss  | DataLoss | 0     
-----------------------------------
9.3 M     Trainable params
0         Non-trainable params
9.3 M     Total params
37.228    Total estimated model params size (MB)


Epoch 9: 100%|██████████| 181/181 [05:01<00:00,  0.60it/s, v_num=16]       

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 181/181 [05:01<00:00,  0.60it/s, v_num=16]


In [51]:
def tensor2image(tensor):
    numpy_image = tensor.detach().cpu().numpy()
    cv2_image = np.transpose(numpy_image, (1, 2, 0))
    cv2_image = cv2.cvtColor(cv2_image, cv2.COLOR_BGR2RGB)
    return cv2.resize(cv2_image, (320, 240))

def plot_images_x(geometry, true_images, predicted_images, error_images, num_images=4):
    fig, axes = plt.subplots(num_images, 4, figsize=(15, 2*num_images))
    fig.tight_layout()

    for i in range(num_images):
        # Plot true image
        axes[i, 0].imshow(geometry[i])
        axes[i, 0].set_title("Geometry")

        # Plot predicted image
        axes[i, 1].imshow(true_images[i][0])
        axes[i, 1].set_title("True U")

        axes[i, 2].imshow(predicted_images[i][0])
        axes[i, 2].set_title("Predicted U")

        axes[i, 3].imshow(error_images[i][0])
        axes[i, 3].set_title("Error U")

        # Remove axis ticks
        axes[i, 0].axis('off')
        axes[i, 1].axis('off')
        axes[i, 2].axis('off')
        axes[i, 3].axis('off')


    plt.show()

def plot_images_y(geometry, true_images, predicted_images, error_images, num_images=4):
    fig, axes = plt.subplots(num_images, 4, figsize=(15, 2*num_images))
    fig.tight_layout()

    for i in range(num_images):
        # Plot true image
        axes[i, 0].imshow(geometry[i])
        axes[i, 0].set_title("Geometry")

        # Plot predicted image
        axes[i, 1].imshow(true_images[i][1])
        axes[i, 1].set_title("True U")

        axes[i, 2].imshow(predicted_images[i][1])
        axes[i, 2].set_title("Predicted U")

        axes[i, 3].imshow(error_images[i][1])
        axes[i, 3].set_title("Error U")


        # Remove axis ticks
        axes[i, 0].axis('off')
        axes[i, 1].axis('off')
        axes[i, 2].axis('off')
        axes[i, 3].axis('off')


    plt.show()

# Assuming true_images and predicted_images are lists of true and predicted images

def run_inference(n = 4):
    gdir = os.path.join(dataset_root, 'Mesh')
    udir = os.path.join(dataset_root, 'W_Data')
    vdir = os.path.join(dataset_root, 'W_Data')

    gfiles = np.random.choice(os.listdir(gdir), size=n, replace=False)
    ufiles = ['U' + f[4:] for f in gfiles]
    vfiles = ['U' + f[4:] for f in gfiles]

    geometry = []
    true_images = []
    predicted_images = []
    error_images = []

    for u, v in zip(ufiles, vfiles):
        u_image_path = os.path.join(udir, u)
        v_image_path = os.path.join(vdir, v)
        uimg = Image.open(u_image_path)
        uimg = uimg.transpose(Image.FLIP_TOP_BOTTOM)
        uimg = uimg.transpose(Image.FLIP_LEFT_RIGHT)
        vimg = Image.open(v_image_path)
        vimg = vimg.transpose(Image.FLIP_TOP_BOTTOM)
        vimg = vimg.transpose(Image.FLIP_LEFT_RIGHT)
        uimg = uimg.resize(((320, 240)))
        vimg = vimg.resize(((320, 240)))
        true_images.append((np.array(uimg), np.array(vimg)))
        

    for g in gfiles:
        image_path = os.path.join(gdir, g)
        img = Image.open(image_path).convert('L')
        geometry.append(transforms.Resize((240, 320))(img))
        input = img_transform(img)
        pred = model.model(input.unsqueeze(0))     
        pred = pred.squeeze(0) 
        pred_image = (tensor2image(pred[0:3, :, :]), tensor2image(pred[3:, :, :]))
        predicted_images.append(pred_image)

    for true, preds in zip(true_images, predicted_images):
        error_u = np.abs(true[0] - preds[0])
        error_v = np.abs(true[1] - preds[1])
        error_images.append((error_u, error_v))
    
    plot_images_x(geometry, true_images, predicted_images, error_images, n)
    plot_images_y(geometry, true_images, predicted_images, error_images, n)


In [4]:
run_inference()

In [13]:
import tensorflow as tf
from tensorflow.keras import layers, models

def unet(input_shape):
    inputs = layers.Input(input_shape)
    
    # Contracting Path
    conv1 = layers.Conv1D(64, 3, activation='relu', padding='same')(inputs)
    pool1 = layers.MaxPooling1D(pool_size=2)(conv1)
    
    conv2 = layers.Conv1D(128, 3, activation='relu', padding='same')(pool1)
    pool2 = layers.MaxPooling1D(pool_size=2)(conv2)
    
    # Expansive Path
    up3 = layers.Conv1D(64, 2, activation='relu', padding='same')(layers.UpSampling1D(size=2)(conv2))
    merge3 = layers.concatenate([conv1, up3], axis=-1)
    conv3 = layers.Conv1D(64, 3, activation='relu', padding='same')(merge3)
    
    # Output Layer
    outputs = layers.Conv1D(1, 1, activation='linear')(conv3)
    
    model = models.Model(inputs=inputs, outputs=outputs)
    return model

# Example usage
input_shape = (100, 1)  # Example input shape for time series data with 100 timesteps and 1 feature
model = unet(input_shape)
model.summary()


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_11      │ (None, 100, 1)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_181 (Conv1D) │ (None, 100, 64)   │        256 │ input_layer_11[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_42    │ (None, 50, 64)    │          0 │ conv1d_181[0][0]  │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_182 (Conv1D) │ (None, 50, 128)   │     24,704 │ max_pooling1d_42… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling1d_32    │ (None, 100, 128)  │          0 │ conv1d_182[0][0]  │
│ (UpSampling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_183 (Conv1D) │ (None, 100, 64)   │     16,448 │ up_sampling1d_32… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_32      │ (None, 100, 128)  │          0 │ conv1d_181[0][0], │
│ (Concatenate)       │                   │            │ conv1d_183[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_184 (Conv1D) │ (None, 100, 64)   │     24,640 │ concatenate_32[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_185 (Conv1D) │ (None, 100, 1)    │         65 │ conv1d_184[0][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 66,113 (258.25 KB)

 Trainable params: 66,113 (258.25 KB)

 Non-trainable params: 0 (0.00 B)

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models

def unet(input_shape):
    inputs = layers.Input(input_shape)
    
    # Contracting Path
    conv1 = layers.Conv1D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv1D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling1D(pool_size=2)(conv1)
    
    conv2 = layers.Conv1D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv1D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling1D(pool_size=2)(conv2)

    conv3 = layers.Conv1D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = layers.Conv1D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = layers.MaxPooling1D(pool_size=2)(conv3)
    
    conv4 = layers.Conv1D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = layers.Conv1D(512, 3, activation='relu', padding='same')(conv4)
    pool4 = layers.MaxPooling1D(pool_size=2)(conv4)
    
    conv5 = layers.Conv1D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = layers.Conv1D(1024, 3, activation='relu', padding='same')(conv5)

    # Expansive Path
    up6 = layers.Conv1D(512, 2, activation='relu', padding='same')(layers.UpSampling1D(size=2)(conv5))
    merge6 = layers.concatenate([conv4, up6], axis=-1)
    conv6 = layers.Conv1D(512, 3, activation='relu', padding='same')(merge6)
    conv6 = layers.Conv1D(512, 3, activation='relu', padding='same')(conv6)
    
    up7 = layers.Conv1D(256, 2, activation='relu', padding='same')(layers.UpSampling1D(size=2)(conv6))
    # Crop conv3 to match the shape of up7
    crop_conv3 = layers.Cropping1D(cropping=(1, 0))(conv3)
    merge7 = layers.concatenate([crop_conv3, up7], axis=-1)
    conv7 = layers.Conv1D(256, 3, activation='relu', padding='same')(merge7)
    conv7 = layers.Conv1D(256, 3, activation='relu', padding='same')(conv7)
    
    up8 = layers.Conv1D(128, 2, activation='relu', padding='same')(layers.UpSampling1D(size=2)(conv7))
    merge8 = layers.concatenate([conv2, up8], axis=-1)
    conv8 = layers.Conv1D(128, 3, activation='relu', padding='same')(merge8)
    conv8 = layers.Conv1D(128, 3, activation='relu', padding='same')(conv8)
    
    up9 = layers.Conv1D(64, 2, activation='relu', padding='same')(layers.UpSampling1D(size=2)(conv8))
    merge9 = layers.concatenate([conv1, up9], axis=-1)
    conv9 = layers.Conv1D(64, 3, activation='relu', padding='same')(merge9)
    conv9 = layers.Conv1D(64, 3, activation='relu', padding='same')(conv9)
    
    # Output Layer
    outputs = layers.Conv1D(1, 1, activation='linear')(conv9)
    
    model = models.Model(inputs=inputs, outputs=outputs)
    return model

# Example usage
input_shape = (100, 1)  # Example input shape for time series data with 100 timesteps and 1 feature
model = unet(input_shape)
model.summary()


In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

def unet(input_shape):
    inputs = layers.Input(input_shape)
    
    # Contracting Path
    conv1 = layers.Conv1D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv1D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling1D(pool_size=2)(conv1)
    
    conv2 = layers.Conv1D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv1D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling1D(pool_size=2)(conv2)

    conv3 = layers.Conv1D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = layers.Conv1D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = layers.MaxPooling1D(pool_size=2)(conv3)
    
    conv4 = layers.Conv1D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = layers.Conv1D(512, 3, activation='relu', padding='same')(conv4)
    pool4 = layers.MaxPooling1D(pool_size=2)(conv4)
    
    conv5 = layers.Conv1D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = layers.Conv1D(1024, 3, activation='relu', padding='same')(conv5)

    # Expansive Path
    up6 = layers.Conv1D(512, 2, activation='relu', padding='same')(layers.UpSampling1D(size=2)(conv5))
    merge6 = layers.concatenate([conv4, up6], axis=-1)
    conv6 = layers.Conv1D(512, 3, activation='relu', padding='same')(merge6)
    conv6 = layers.Conv1D(512, 3, activation='relu', padding='same')(conv6)
    
    up7 = layers.Conv1D(256, 2, activation='relu', padding='same')(layers.UpSampling1D(size=2)(conv6))
    crop_conv3 = layers.Cropping1D(cropping=(1, 0))(conv3)
    merge7 = layers.concatenate([crop_conv3, up7], axis=-1)
    conv7 = layers.Conv1D(256, 3, activation='relu', padding='same')(merge7)
    conv7 = layers.Conv1D(256, 3, activation='relu', padding='same')(conv7)
    
    up8 = layers.Conv1D(128, 2, activation='relu', padding='same')(layers.UpSampling1D(size=2)(conv7))
    crop_conv2 = layers.Cropping1D(cropping=(1, 0))(conv2)
    merge8 = layers.concatenate([crop_conv2, up8], axis=-1)
    conv8 = layers.Conv1D(128, 3, activation='relu', padding='same')(merge8)
    conv8 = layers.Conv1D(128, 3, activation='relu', padding='same')(conv8)
    
    up9 = layers.Conv1D(64, 2, activation='relu', padding='same')(layers.UpSampling1D(size=2)(conv8))
    crop_conv1 = layers.Cropping1D(cropping=(1, 0))(conv1)
    merge9 = layers.concatenate([crop_conv1, up9], axis=-1)
    conv9 = layers.Conv1D(64, 3, activation='relu', padding='same')(merge9)
    conv9 = layers.Conv1D(64, 3, activation='relu', padding='same')(conv9)
    
    # Output Layer
    outputs = layers.Conv1D(1, 1, activation='linear')(conv9)
    
    model = models.Model(inputs=inputs, outputs=outputs)
    return model

# Example usage
input_shape = (100, 1)  # Example input shape for time series data with 100 timesteps and 1 feature
model = unet(input_shape)
model.summary()


In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models

def unet(input_shape):
    inputs = layers.Input(input_shape)
    
    # Contracting Path
    conv1 = layers.Conv1D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv1D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling1D(pool_size=2)(conv1)
    
    conv2 = layers.Conv1D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv1D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling1D(pool_size=2)(conv2)

    conv3 = layers.Conv1D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = layers.Conv1D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = layers.MaxPooling1D(pool_size=2)(conv3)
    
    conv4 = layers.Conv1D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = layers.Conv1D(512, 3, activation='relu', padding='same')(conv4)
    pool4 = layers.MaxPooling1D(pool_size=2)(conv4)
    
    conv5 = layers.Conv1D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = layers.Conv1D(1024, 3, activation='relu', padding='same')(conv5)

    # Expansive Path
    up6 = layers.Conv1D(512, 2, activation='relu', padding='same')(layers.UpSampling1D(size=2)(conv5))
    merge6 = layers.concatenate([conv4, up6], axis=-1)
    conv6 = layers.Conv1D(512, 3, activation='relu', padding='same')(merge6)
    conv6 = layers.Conv1D(512, 3, activation='relu', padding='same')(conv6)
    
    up7 = layers.Conv1D(256, 2, activation='relu', padding='same')(layers.UpSampling1D(size=2)(conv6))
    merge7 = layers.concatenate([conv3[:, :-1], up7], axis=-1)  # Crop conv3 before concatenating
    conv7 = layers.Conv1D(256, 3, activation='relu', padding='same')(merge7)
    conv7 = layers.Conv1D(256, 3, activation='relu', padding='same')(conv7)
    
    up8 = layers.Conv1D(128, 2, activation='relu', padding='same')(layers.UpSampling1D(size=2)(conv7))
    merge8 = layers.concatenate([conv2[:, :-1], up8], axis=-1)  # Crop conv2 before concatenating
    conv8 = layers.Conv1D(128, 3, activation='relu', padding='same')(merge8)
    conv8 = layers.Conv1D(128, 3, activation='relu', padding='same')(conv8)
    
    up9 = layers.Conv1D(64, 2, activation='relu', padding='same')(layers.UpSampling1D(size=2)(conv8))
    merge9 = layers.concatenate([conv1[:, :-1], up9], axis=-1)  # Crop conv1 before concatenating
    conv9 = layers.Conv1D(64, 3, activation='relu', padding='same')(merge9)
    conv9 = layers.Conv1D(64, 3, activation='relu', padding='same')(conv9)
    
    # Output Layer
    outputs = layers.Conv1D(1, 1, activation='linear')(conv9)
    
    model = models.Model(inputs=inputs, outputs=outputs)
    return model

# Example usage
input_shape = (100, 1)  # Example input shape for time series data with 100 timesteps and 1 feature
model = unet(input_shape)
model.summary()


In [12]:
import tensorflow as tf
from tensorflow.keras import layers, models

def unet(input_shape):
    inputs = layers.Input(input_shape)
    
    # Contracting Path
    conv1 = layers.Conv1D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv1D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling1D(pool_size=2)(conv1)
    
    conv2 = layers.Conv1D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv1D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling1D(pool_size=2)(conv2)

    conv3 = layers.Conv1D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = layers.Conv1D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = layers.MaxPooling1D(pool_size=2)(conv3)
    
    conv4 = layers.Conv1D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = layers.Conv1D(512, 3, activation='relu', padding='same')(conv4)
    pool4 = layers.MaxPooling1D(pool_size=2)(conv4)
    
    conv5 = layers.Conv1D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = layers.Conv1D(1024, 3, activation='relu', padding='same')(conv5)

    # Expansive Path
    up6 = layers.Conv1D(512, 2, activation='relu', padding='same')(layers.UpSampling1D(size=2)(conv5))
    merge6 = layers.concatenate([conv4, up6], axis=-1)
    conv6 = layers.Conv1D(512, 3, activation='relu', padding='same')(merge6)
    conv6 = layers.Conv1D(512, 3, activation='relu', padding='same')(conv6)
    
    up7 = layers.Conv1D(256, 2, activation='relu', padding='same')(layers.UpSampling1D(size=2)(conv6))
    merge7 = layers.concatenate([conv3[:, :-1], up7], axis=-1)  # Crop conv3 before concatenating
    conv7 = layers.Conv1D(256, 3, activation='relu', padding='same')(merge7)
    conv7 = layers.Conv1D(256, 3, activation='relu', padding='same')(conv7)
    
    up8 = layers.Conv1D(128, 2, activation='relu', padding='same')(layers.UpSampling1D(size=2)(conv7))
    merge8 = layers.concatenate([conv2[:, :-2], up8], axis=-1)  # Crop conv2 before concatenating
    conv8 = layers.Conv1D(128, 3, activation='relu', padding='same')(merge8)
    conv8 = layers.Conv1D(128, 3, activation='relu', padding='same')(conv8)
    
    up9 = layers.Conv1D(64, 2, activation='relu', padding='same')(layers.UpSampling1D(size=2)(conv8))
    merge9 = layers.concatenate([conv1[:, :-4], up9], axis=-1)  # Crop conv1 before concatenating
    conv9 = layers.Conv1D(64, 3, activation='relu', padding='same')(merge9)
    conv9 = layers.Conv1D(64, 3, activation='relu', padding='same')(conv9)
    
    # Output Layer
    outputs = layers.Conv1D(1, 1, activation='linear')(conv9)
    
    model = models.Model(inputs=inputs, outputs=outputs)
    return model

# Example usage
input_shape = (100, 1)  # Example input shape for time series data with 100 timesteps and 1 feature
model = unet(input_shape)
model.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_10      │ (None, 100, 1)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_158 (Conv1D) │ (None, 100, 64)   │        256 │ input_layer_10[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_159 (Conv1D) │ (None, 100, 64)   │     12,352 │ conv1d_158[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_38    │ (None, 50, 64)    │          0 │ conv1d_159[0][0]  │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_160 (Conv1D) │ (None, 50, 128)   │     24,704 │ max_pooling1d_38… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_161 (Conv1D) │ (None, 50, 128)   │     49,280 │ conv1d_160[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_39    │ (None, 25, 128)   │          0 │ conv1d_161[0][0]  │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_162 (Conv1D) │ (None, 25, 256)   │     98,560 │ max_pooling1d_39… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_163 (Conv1D) │ (None, 25, 256)   │    196,864 │ conv1d_162[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_40    │ (None, 12, 256)   │          0 │ conv1d_163[0][0]  │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_164 (Conv1D) │ (None, 12, 512)   │    393,728 │ max_pooling1d_40… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_165 (Conv1D) │ (None, 12, 512)   │    786,944 │ conv1d_164[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_41    │ (None, 6, 512)    │          0 │ conv1d_165[0][0]  │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_166 (Conv1D) │ (None, 6, 1024)   │  1,573,888 │ max_pooling1d_41… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_167 (Conv1D) │ (None, 6, 1024)   │  3,146,752 │ conv1d_166[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling1d_28    │ (None, 12, 1024)  │          0 │ conv1d_167[0][0]  │
│ (UpSampling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_168 (Conv1D) │ (None, 12, 512)   │  1,049,088 │ up_sampling1d_28… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_28      │ (None, 12, 1024)  │          0 │ conv1d_165[0][0], │
│ (Concatenate)       │                   │            │ conv1d_168[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_169 (Conv1D) │ (None, 12, 512)   │  1,573,376 │ concatenate_28[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_170 (Conv1D) │ (None, 12, 512)   │    786,944 │ conv1d_169[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling1d_29    │ (None, 24, 512)   │          0 │ conv1d_170[0][0]  │
│ (UpSampling1D)      │                   │            │                 

 Total params: 10,812,353 (41.25 MB)

 Trainable params: 10,812,353 (41.25 MB)

 Non-trainable params: 0 (0.00 B)